In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Path to your dataset
train_data_dir = r"C:\Users\anmol\OneDrive - UPES\Documents\UPES\Semester 7\Major 1\archive\Alzheimer_s Dataset\train"
test_data_dir = r"C:\Users\anmol\OneDrive - UPES\Documents\UPES\Semester 7\Major 1\archive\Alzheimer_s Dataset\test"

In [3]:
# Define data generators with normalization and augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    brightness_range=[0.7, 1.3]
)

In [4]:
# Create data generators for training and testing
train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 5121 images belonging to 4 classes.
Found 1270 images belonging to 4 classes.


In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D

def dense_block(x, blocks, filters):
    for i in range(blocks):
        x = conv_block(x, filters)
    return x

def transition_block(x, filters):
    x = Conv2D(filters, (1, 1), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    return x

def conv_block(x, filters):
    x = Conv2D(filters, (1, 1), activation='relu', padding='same')(x)
    x = Conv2D(filters, (3, 3), activation='relu', padding='same')(x)
    return x

In [6]:
def build_densenet121(input_shape=(224, 224, 3)):
    input_tensor = Input(shape=input_shape)

    # Initial Convolutional Layer
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu', name='Conv_1')(input_tensor)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # Dense Blocks
    x = dense_block(x, 6, 128)  # Conv_2
    x = transition_block(x, 256)  # Conv_3
    x = dense_block(x, 12, 512)  # Conv_4
    x = transition_block(x, 1024)  # Conv_5
    x = dense_block(x, 32, 1024)  # Output of Conv_5

    # Global Average Pooling
    x = GlobalAveragePooling2D()(x)

    # Build model
    model = Model(inputs=input_tensor, outputs=x, name='densenet121_feature_extractor')
    return model

In [7]:
# Create DenseNet121 model for feature extraction
densenet121_model = build_densenet121()

# Display the model summary
densenet121_model.summary()

Model: "densenet121_feature_extractor"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
Conv_1 (Conv2D)              (None, 112, 112, 64)      9472      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d (Conv2D)              (None, 56, 56, 128)       8320      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 128)       147584    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 128)       16512     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 5

In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D

def dense_block(x, blocks, filters):
    for i in range(blocks):
        x = conv_block(x, filters)
    return x

def transition_block(x, filters):
    x = Conv2D(filters, (1, 1), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    return x

def conv_block(x, filters):
    x = Conv2D(filters, (1, 1), activation='relu', padding='same')(x)
    x = Conv2D(filters, (3, 3), activation='relu', padding='same')(x)
    return x

In [9]:
def build_densenet201(input_shape=(224, 224, 3)):
    input_tensor = Input(shape=input_shape)

    # Initial Convolutional Layer
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu', name='Conv_1')(input_tensor)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # Dense Blocks
    x = dense_block(x, 12, 128)  # Conv_2
    x = transition_block(x, 512)  # Conv_3
    x = dense_block(x, 24, 896)  # Conv_4
    x = transition_block(x, 1920)  # Conv_5
    x = dense_block(x, 64, 1920)  # Output of Conv_5

    # Global Average Pooling
    x = GlobalAveragePooling2D()(x)

    # Build model
    model = Model(inputs=input_tensor, outputs=x, name='densenet201_feature_extractor')
    return model

In [10]:
# Create DenseNet201 model for feature extraction
densenet201_model = build_densenet201()

# Display the model summary
densenet201_model.summary()

Model: "densenet201_feature_extractor"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
Conv_1 (Conv2D)              (None, 112, 112, 64)      9472      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_102 (Conv2D)          (None, 56, 56, 128)       8320      
_________________________________________________________________
conv2d_103 (Conv2D)          (None, 56, 56, 128)       147584    
_________________________________________________________________
conv2d_104 (Conv2D)          (None, 56, 56, 128)       16512     
_________________________________________________________________
conv2d_105 (Conv2D)          (None, 5

c:\ProgramData\anaconda3\envs\myenv\lib\site-packages\tensorflow\python\keras\utils\layer_utils.py:109: RuntimeWarning: overflow encountered in long_scalars
  return int(sum(np.prod(p) for p in standardized_weight_shapes))


In [13]:
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from tensorflow.keras.layers import Concatenate, Dense

In [15]:
# Extract features from Block-5 of DenseNet121
densenet121_output = densenet121_model.get_layer('conv2d_101').output  # Adjust layer name if needed

# Global Average Pooling
x = GlobalAveragePooling2D()(densenet121_output)

# Additional Dense Layer
x = Dense(1024, activation='relu', name='dense_1')(x)

# Machine Learning Classifiers
svm_output = Dense(1, activation='sigmoid', name='svm_output')(x)
naive_bayes_output = Dense(1, activation='sigmoid', name='naive_bayes_output')(x)
xgboost_output = Dense(1, activation='sigmoid', name='xgboost_output')(x)

# Concatenate classifier outputs
classifier_outputs = Concatenate(name='classifier_outputs')([svm_output, naive_bayes_output, xgboost_output])

# Final Dense Layer
final_output = Dense(1, activation='sigmoid', name='final_output')(classifier_outputs)

# Build the hybrid model
hybrid_model_densenet121 = Model(inputs=densenet121_model.input, outputs=final_output, name='hybrid_densenet121_model')

# Display the model summary
hybrid_model_densenet121.summary()

Model: "hybrid_densenet121_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv_1 (Conv2D)                 (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 56, 56, 64)   0           Conv_1[0][0]                     
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 56, 56, 128)  8320        max_pooling2d[0][0]              
___________________________________________________________________________

In [17]:
# Extract features from Block-5 of DenseNet201
densenet201_output = densenet201_model.get_layer('conv2d_303').output  # Adjust layer name if needed

# Global Average Pooling
x = GlobalAveragePooling2D()(densenet201_output)

# Additional Dense Layer
x = Dense(1920, activation='relu', name='dense_1')(x)

# Machine Learning Classifiers
svm_output = Dense(1, activation='sigmoid', name='svm_output')(x)
naive_bayes_output = Dense(1, activation='sigmoid', name='naive_bayes_output')(x)
xgboost_output = Dense(1, activation='sigmoid', name='xgboost_output')(x)

# Concatenate classifier outputs
classifier_outputs = Concatenate(name='classifier_outputs')([svm_output, naive_bayes_output, xgboost_output])

# Final Dense Layer
final_output = Dense(1, activation='sigmoid', name='final_output')(classifier_outputs)

# Build the hybrid model
hybrid_model_densenet201 = Model(inputs=densenet201_model.input, outputs=final_output, name='hybrid_densenet201_model')

# Display the model summary
hybrid_model_densenet201.summary()

Model: "hybrid_densenet201_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv_1 (Conv2D)                 (None, 112, 112, 64) 9472        input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_3 (MaxPooling2D)  (None, 56, 56, 64)   0           Conv_1[0][0]                     
__________________________________________________________________________________________________
conv2d_102 (Conv2D)             (None, 56, 56, 128)  8320        max_pooling2d_3[0][0]            
___________________________________________________________________________

c:\ProgramData\anaconda3\envs\myenv\lib\site-packages\tensorflow\python\keras\utils\layer_utils.py:109: RuntimeWarning: overflow encountered in long_scalars
  return int(sum(np.prod(p) for p in standardized_weight_shapes))


In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Extract features from Block-5 of DenseNet121 for training and testing
densenet121_output_train = densenet121_model.predict(train_generator)
densenet121_output_test = densenet121_model.predict(test_generator)

# Extract features from Block-5 of DenseNet201 for training and testing
densenet201_output_train = densenet201_model.predict(train_generator)
densenet201_output_test = densenet201_model.predict(test_generator)

# Combine features from both models
X_train_combined = np.concatenate((densenet121_output_train, densenet201_output_train), axis=1)
X_test_combined = np.concatenate((densenet121_output_test, densenet201_output_test), axis=1)

# Labels for training data
y_train = train_generator.classes